In [17]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [9]:
df = pd.read_csv('data.csv', sep=';')

In [26]:
df.sample(10)

,Cod_cmd,Libellé produit,Vendeur,Univers,Nature,Date de commande,Montant cmd,Quantité,Prix transport,Délai transport annoncé
476492,181217541,Matelas ressorts 140x190 cm,Vendeur 1,Bureau Rangement,Meuble à chaussures,03/01/2021 00:00,1 148,1,"24,17",NaN
244042,184970381,Tapis grafic bleu doré 160x230cm,Autre vendeur,Décoration Textile,Tapis de Salon et Ch,22/02/2021 00:00,131,1,0,4.0
497960,183149730,Meuble haut 80 cm 2 portes,Vendeur 1,Cuisine Salle de bain,Meuble haut cuisine,30/01/2021 00:00,15,1,0,NaN
523356,211101833,Expresso portionné,Vendeur 1,Petit Electroménager,Expresso,31/03/2021 00:00,61,1,"6,66",4.0
167710,185414853,Placard de bureau en métal meuble de rangement...,Autre vendeur,Bureau Rangement,Rangement bureau,02/03/2021 00:00,163,1,0,8.0
468405,181471057,Canapé d angle convertible réversible 4 places...,Vendeur 1,Canapé Salon Séjour,Canapé d'angle,07/01/2021 00:00,802,1,0,NaN
263900,185728152,Bibliothèque 6 cases,Vendeur 1,Bureau Rangement,Bibliotheque,07/03/2021 00:00,75,1,0,0.0
462917,181311058,Miroir 52 cm,Vendeur 1,Décoration Textile,Miroir,05/01/2021 00:00,21,1,0,NaN
513477,186481160,Armoire 4 portes,Vendeur 1,Chambre Literie,Armoire,20/03/2021 00:00,283,1,0,0.0
68769,186035998,Ensemble table et chaises contemporain ch ne c...,Autre vendeur,Cuisine Salle de bain,Ens table chaises,13/03/2021 00:00,290,1,"16,58",33.0


In [29]:
df['Cod_cmd'].value_counts().count()

404939

In [31]:
# Get the most common Univers and Nature for each product
product_categories = df.groupby('Libellé produit').agg({
    'Univers': lambda x: x.mode().iat[0] if not x.mode().empty else None,
    'Nature': lambda x: x.mode().iat[0] if not x.mode().empty else None
}).reset_index()

df_unique_products = product_categories[['Libellé produit', 'Univers', 'Nature']]

print("Number of unique products:", len(df_unique_products))
df_unique_products.head()


Number of unique products: 56018


,Libellé produit,Univers,Nature
0,40 742 ensemble 5 en 1 tondeuse et regle barb...,Petit Electroménager,Tondeuse multistyle
1,60cm fronton applique mural style arbre de vi...,Décoration Textile,Objet deco
2,9 vip pets vague 1,Enfant Bébé,Poupée et poupon
3,agère murale hwc h37 étagère suspendue tiroir...,Canapé Salon Séjour,Etagère
4,allan canapé 3 places convertible ouverture e...,Petit Electroménager,Friteuse


In [32]:
df_unique_products.to_csv('unique_products.csv', index=False)

In [34]:
df = pd.read_csv('unique_products.csv')
df.sample(10)

,Libellé produit,Univers,Nature
32485,Meuble a chaussures boots meuble a chaussures ...,Bureau Rangement,Meuble à chaussures
7790,Cadre de lit en métal robuste sommier à lattes...,Chambre Literie,Lit adulte
49063,Table à manger extensible renee,Canapé Salon Séjour,Table
48163,Table de bar avec rangement table bistro haut ...,Canapé Salon Séjour,Table
43162,Sa790141 joint 4 5 6l 220mm pour autocuiseur seb,Petit Electroménager,Pièces détachées
43361,Salon de jardin cordoue en résine tressée gris...,Jardin Loisirs Sport,Salon de jardin
21522,Icaverne etagères et casiers à chaussures gamm...,Bureau Rangement,Porte chaussures
4283,Auna krcd 150 radio encastrable lecteur cd usb...,TV Son Multimédia,Radio reveil
20579,Hotte décorative murale 90cm 55db 721m3 h inox...,Gros Electroménager,Hotte
14035,Domy table basse relevable extensible papier d...,Canapé Salon Séjour,Table basse


In [36]:
df['Nature'].value_counts().count()

594

In [ ]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tokenizers", "scikit-learn"])


In [48]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import os

# Ensure df is loaded (from earlier cells). Expect columns: 'Libellé produit', 'Nature'
assert 'df' in globals(), "DataFrame df must be loaded earlier."
assert {'Libellé produit','Nature'}.issubset(df.columns), "df must have 'Libellé produit' and 'Nature' columns"

# Use the deduplicated products if available
if 'df_unique_products' in globals():
    data_df = df_unique_products.rename(columns={'Libellé produit': 'label', 'Nature': 'target'})
else:
    data_df = df.rename(columns={'Libellé produit': 'label', 'Nature': 'target'})[['label','target']].dropna()

# Basic cleaning: drop rows with missing or empty labels/targets
data_df = data_df.dropna(subset=['label','target'])
data_df = data_df[(data_df['label'].astype(str).str.strip()!='') & (data_df['target'].astype(str).str.strip()!='')]

# Prepare corpus for BPE
corpus = data_df['label'].astype(str).tolist()

# Initialize BPE tokenizer
bpe_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
bpe_tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
bpe_tokenizer.pre_tokenizer = Whitespace()

# Special tokens and training
special_tokens = ["[PAD]", "[UNK]"]
trainer = BpeTrainer(vocab_size=20000, min_frequency=2, special_tokens=special_tokens)

bpe_tokenizer.train_from_iterator(corpus, trainer=trainer)

# Post-processor for consistent BOS/EOS if desired (not necessary for bag-of-tokens)
bpe_tokenizer.post_processor = TemplateProcessing(
    single="$0",
    pair="$A $B",
    special_tokens=[("[PAD]", 0), ("[UNK]", 1)]
)

# Save to disk to reuse later
bpe_dir = "./artifacts"
os.makedirs(bpe_dir, exist_ok=True)
bpe_path = os.path.join(bpe_dir, "bpe_tokenizer.json")
bpe_tokenizer.save(bpe_path)

print(f"Trained BPE tokenizer with vocab size: {bpe_tokenizer.get_vocab_size()} and saved to {bpe_path}")





Trained BPE tokenizer with vocab size: 20000 and saved to ./artifacts/bpe_tokenizer.json


In [ ]:
# Build vocabulary mapping from tokenizer
vocab = bpe_tokenizer.get_vocab()
# Invert mapping to get id->token if needed
id_to_token = {idx: tok for tok, idx in vocab.items()}

# Helper: encode to BPE token ids, ignoring specials
SPECIAL_IDS = {vocab.get("[PAD]"), vocab.get("[UNK]")}

def bpe_tokenize(text: str):
    if not isinstance(text, str):
        text = str(text) if text is not None else ""
    encoding = bpe_tokenizer.encode(text)
    # Return tokens (strings) excluding specials
    tokens = [t for t in encoding.tokens if vocab.get(t) not in SPECIAL_IDS]
    return tokens

# Create bag-of-BPE-token counts
from collections import Counter

all_tokens = []
for t in data_df['label'].tolist():
    all_tokens.extend(bpe_tokenize(t))

# Select top-k tokens to form features
from collections import Counter
freqs = Counter(all_tokens)
max_features = 50000
most_common_tokens = [tok for tok, _ in freqs.most_common(max_features)]
feature_index = {tok: i for i, tok in enumerate(most_common_tokens)}

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, vstack

rows, cols, vals = [], [], []
for row_idx, text in enumerate(data_df['label'].tolist()):
    counts = Counter(bpe_tokenize(text))
    for tok, c in counts.items():
        col_idx = feature_index.get(tok)
        if col_idx is not None:
            rows.append(row_idx)
            cols.append(col_idx)
            vals.append(c)

X = csr_matrix((vals, (rows, cols)), shape=(len(data_df), len(feature_index)), dtype=np.float64)
y = data_df['target'].astype(str).values

# Robust split handling rare classes (count < 2)
counts = pd.Series(y).value_counts()
freq_classes = counts[counts >= 2].index
frequent_mask = np.isin(y, freq_classes)
idx_frequent = np.where(frequent_mask)[0]
idx_rare = np.where(~frequent_mask)[0]

if len(idx_frequent) >= 2:
    X_f = X[idx_frequent]
    y_f = y[idx_frequent]
    try:
        X_train_f, X_test, y_train_f, y_test = train_test_split(
            X_f, y_f, test_size=0.1, random_state=42, stratify=y_f
        )
    except ValueError:
        X_train_f, X_test, y_train_f, y_test = train_test_split(
            X_f, y_f, test_size=0.1, random_state=42, shuffle=True, stratify=None
        )
    # Add all rare-class samples to training set
    if len(idx_rare) > 0:
        X_train = vstack([X_train_f, X[idx_rare]])
        y_train = np.concatenate([y_train_f, y[idx_rare]])
    else:
        X_train, y_train = X_train_f, y_train_f
else:
    # Fallback: no stratification possible
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42, shuffle=True, stratify=None
    )

# Fit Multinomial Naive Bayes
nb = MultinomialNB(alpha=0.1)
nb.fit(X_train, y_train)

print('Train size:', X_train.shape, 'Test size:', X_test.shape)
print('Classes:', nb.classes_.shape)
print('Test accuracy:', nb.score(X_test, y_test))
print(classification_report(y_test, nb.predict(X_test)))


In [50]:
from typing import Dict

# Prepare structures for prediction
class_names = nb.classes_.tolist()
class_index = {c: i for i, c in enumerate(class_names)}


def vectorize_labels(texts):
    rows, cols, vals = [], [], []
    for row_idx, text in enumerate(texts):
        counts = Counter(bpe_tokenize(text))
        for tok, c in counts.items():
            col_idx = feature_index.get(tok)
            if col_idx is not None:
                rows.append(row_idx)
                cols.append(col_idx)
                vals.append(c)
    return csr_matrix((vals, (rows, cols)), shape=(len(texts), len(feature_index)), dtype=np.float64)


def predict_nature_proba(label_text: str) -> Dict[str, float]:
    Xq = vectorize_labels([label_text])
    proba = nb.predict_proba(Xq)[0]
    return {cls: float(prob) for cls, prob in zip(class_names, proba)}

# Example usage
example_label = data_df['label'].iloc[0]
probs = predict_nature_proba(example_label)
print(example_label)
# Display top-10 predicted Natures
for cls, p in sorted(probs.items(), key=lambda kv: kv[1], reverse=True)[:10]:
    print(f"{cls}: {p:.4f}")


 40 742 ensemble 5 en 1 tondeuse et regle barbe rechargeable
Tondeuse multistyle: 0.9889
Coiffeuse: 0.0053
Accessoire pem: 0.0015
Accessoire ch adulte: 0.0010
Petit meuble séjour: 0.0007
Ens table chaises: 0.0006
Evier: 0.0003
Chevet: 0.0003
Bar: 0.0002
Rangement bureau: 0.0001


In [ ]:
import numpy as np
import pandas as pd

# Train on full data to score items
nb_full = MultinomialNB(alpha=0.1)
nb_full.fit(X, y)

class_names_full = nb_full.classes_.tolist()


def score_items_and_find_mislabels(
    df_labels_targets: pd.DataFrame,
    prob_threshold: float = 0.2,
    topk: int = 3,
) -> pd.DataFrame:
    """
    Identify potential mislabelled items by scoring each label with the trained model.

    - prob_threshold: if model's probability for the current label's target is below this, flag as candidate.
    - topk: include top-k alternative class suggestions and their probabilities.
    """
    texts = df_labels_targets['label'].astype(str).tolist()
    X_all = vectorize_labels(texts)
    proba_all = nb_full.predict_proba(X_all)
    pred_all = nb_full.predict(X_all)

    # Map true target index in classes
    class_index_full = {c: i for i, c in enumerate(class_names_full)}

    rows = []
    for i, (label_text, true_cls) in enumerate(zip(df_labels_targets['label'], df_labels_targets['target'])):
        probs = proba_all[i]
        true_idx = class_index_full.get(str(true_cls), None)
        true_prob = float(probs[true_idx]) if true_idx is not None else np.nan
        top_indices = np.argsort(probs)[::-1][:topk]
        top_suggestions = [(class_names_full[j], float(probs[j])) for j in top_indices]
        is_mislabel = (not np.isnan(true_prob)) and (true_prob < prob_threshold)
        rows.append({
            'label': label_text,
            'true_nature': true_cls,
            'true_prob': true_prob,
            'pred_nature': pred_all[i],
            'pred_prob': float(np.max(probs)),
            'top_suggestions': top_suggestions,
            'candidate_mislabel': bool(is_mislabel),
        })

    result = pd.DataFrame(rows)
    return result.sort_values(['candidate_mislabel','true_prob'], ascending=[False, True]).reset_index(drop=True)


mislabel_df = score_items_and_find_mislabels(data_df, prob_threshold=0.2, topk=5)
print('Candidates:', mislabel_df['candidate_mislabel'].sum(), 'of', len(mislabel_df))
mislabel_path = './artifacts/mislabel_candidates.csv'
mislabel_df.to_csv(mislabel_path, index=False)
print('Saved to', mislabel_path)

# Show a preview of flagged items
mislabel_df[mislabel_df['candidate_mislabel']].head(10)
